<a href="https://colab.research.google.com/github/prikshitkverma/Gemma_fine_tuning/blob/main/gemma_3_1b_it_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================
# 1. INSTALL LIBRARIES
# ============================================
#!pip uninstall -y torch torchvision torchaudio transformers trl accelerate datasets huggingface_hub
#!pip install -q torch==2.6.0+cu124 torchvision==0.21.0+cu124 torchaudio==2.6.0+cu124 --index-url https://download.pytorch.org/whl/cu124
#!pip install -q transformers==4.45.2 trl==0.11.6 accelerate==1.1.1 datasets==3.1.0 huggingface_hub==0.28.1 sentencepiece pyarrow==18.0.0 evaluate tensorboard
!pip uninstall -y torch torchvision torchaudio transformers trl accelerate datasets huggingface_hub
!pip install -q torch==2.6.0+cu124 torchvision==0.21.0+cu124 torchaudio==2.6.0+cu124 --index-url https://download.pytorch.org/whl/cu124
!pip install -q transformers==4.45.2 trl==0.11.6 accelerate==1.1.1 datasets==3.1.0 huggingface_hub==0.28.1 sentencepiece pyarrow==18.0.0
!pip install -q datasets trl sentencepiece huggingface_hub
!pip install evaluate




In [ ]:
# ============================================
# 2. SETUP AND AUTHENTICATION
# ============================================
import torch
import os
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    pipeline,
)
from trl import SFTTrainer, SFTConfig
from huggingface_hub import login
import evaluate

# Login to Hugging Face
HF_TOKEN = "hf_token"
login(token=HF_TOKEN)

In [ ]:
# ============================================
# 3. CONFIGURE MODEL AND DIRECTORIES
# ============================================
base_model = "google/gemma-3-1b-it"
output_dir = "./gemma-natural-farming-qa"


# ============================================
# 4. LOAD AND PREPARE THE DATASET
# ============================================

from datasets import load_dataset

data_file = "/content/nf_datasett.jsonl"

# Load dataset
dataset = load_dataset(
    "json",
    data_files=data_file,
    split="train"
)

# Shuffle
dataset = dataset.shuffle(seed=42)

# 80% train, 10% val, 10% test
train_test = dataset.train_test_split(test_size=0.2)
val_test = train_test["test"].train_test_split(test_size=0.5)

dataset_dict = {
    "train": train_test["train"],
    "validation": val_test["train"],
    "test": val_test["test"]
}

# Inspect dataset
print("✅ Dataset Split Summary:")
print(
    f"Train: {len(dataset_dict['train'])} | "
    f"Validation: {len(dataset_dict['validation'])} | "
    f"Test: {len(dataset_dict['test'])}"
)

print("\nExample data sample:")
print(dataset_dict["train"][0]["messages"])


In [ ]:
# ============================================
# 5. LOAD MODEL AND TOKENIZER
# ============================================
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    torch_dtype="auto",
    device_map="auto",
    attn_implementation="eager"
)
tokenizer = AutoTokenizer.from_pretrained(base_model)
print(f"✅ Model loaded on {model.device} | dtype: {model.dtype}")

In [ ]:
# ============================================
# 6. CONFIGURE THE TRAINING PROCESS
# ============================================
sft_config = SFTConfig(
    output_dir=output_dir,
    num_train_epochs=3,
    # max_seq_length=256, # Removed max_seq_length
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    optim="adamw_torch_fused",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-5,
    lr_scheduler_type="cosine",
    bf16=torch.cuda.is_bf16_supported(),
    fp16=False,
    push_to_hub=False,
    report_to="tensorboard"
)


In [ ]:
# ============================================
# 7. TRAIN AND VALIDATE MODEL
# ============================================
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=dataset_dict["train"],
    eval_dataset=dataset_dict["validation"],
    processing_class=tokenizer,
)

print("🚀 Starting fine-tuning...")
trainer.train()

print("💾 Saving final model...")
trainer.save_model(output_dir)



In [ ]:
!pip install rouge_score

In [ ]:

# ============================================
# 8. VALIDATE MODEL PERFORMANCE
# ============================================
print("\n🔍 Validating model on validation set...")
model.eval()

# Use a simple text-generation pipeline
val_pipe = pipeline("text-generation", model=output_dir, tokenizer=tokenizer)

bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

generated_texts = []
reference_texts = []

for i, sample in enumerate(dataset_dict["validation"]):
    user_msg = [{"role": "user", "content": sample["messages"][0]["content"]}]
    prompt = tokenizer.apply_chat_template(user_msg, tokenize=False, add_generation_prompt=True)
    output = val_pipe(prompt, max_new_tokens=128, num_return_sequences=1)[0]["generated_text"][len(prompt):].strip()
    generated_texts.append(output)
    reference_texts.append(sample["messages"][1]["content"])
    if i < 3:
        print(f"\nExample {i+1}:")
        print(f"Q: {sample['messages'][0]['content']}")
        print(f"Model: {output}")
        print(f"Ref: {sample['messages'][1]['content']}")

# Compute metrics
bleu_score = bleu.compute(predictions=generated_texts, references=reference_texts)
rouge_score = rouge.compute(predictions=generated_texts, references=reference_texts)

print("\n📊 Validation Metrics:")
print(f"BLEU Score: {bleu_score['bleu']:.4f}")
print(f"ROUGE-L: {rouge_score['rougeL']:.4f}")

In [ ]:

!pip install bert_score
bertscore = evaluate.load("bertscore")
results = bertscore.compute(predictions=generated_texts, references=reference_texts, lang="en")
print(sum(results["f1"]) / len(results["f1"]))

In [ ]:
import math

def compute_perplexity(model, tokenizer, dataset):
    model.eval()
    losses = []

    for sample in dataset:
        text = tokenizer.apply_chat_template(
            sample["messages"],
            tokenize=False
        )
        inputs = tokenizer(text, return_tensors="pt", truncation=True).to(model.device)

        with torch.no_grad():
            outputs = model(**inputs, labels=inputs["input_ids"])
            losses.append(outputs.loss.item())

    mean_loss = sum(losses) / len(losses)
    return math.exp(mean_loss)

ppl = compute_perplexity(model, tokenizer, dataset_dict["validation"])
print(f"\n📉 Validation Perplexity: {ppl:.2f}")


In [ ]:
# ============================================
# 9. TEST INTERACTIVELY
# ============================================
print("\n--- Interactive Testing ---")
test_pipe = pipeline("text-generation", model=output_dir, tokenizer=tokenizer)

while True:
    question = input("\nEnter your question (or type 'exit' to quit): ").strip()
    if question.lower() == "exit":
        print("👋 Exiting...")
        break

    messages = [{"role": "user", "content": question}]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    outputs = test_pipe(prompt, max_new_tokens=256)
    print(f"\n🧠 Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")
